<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/TransferLaerning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
 #Get data (10% of 10 food classes from Food101) - https://www.kaggle.com/dansbecker/food-101 
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-05-17 16:53:26--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.13.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.13.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.2’

10_food_classes_10_ 100%[===================>] 160.74M   233MB/s    in 0.7s    

2022-05-17 16:53:27 (233 MB/s) - ‘10_food_classes_10_percent.zip.2’ saved [168546183/168546183]



In [12]:
import os 
for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images is '{dirpath}'")

There are 2 directories and 0 images is '10_food_classes_10_percent'
There are 10 directories and 0 images is '10_food_classes_10_percent/test'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/ramen'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/steak'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/hamburger'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/sushi'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/chicken_wings'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/grilled_salmon'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/fried_rice'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/ice_cream'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/pizza'
There are 0 directories and 250 images is '10_food_classes_10_percent/test/chicken_curry

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"
train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)
train_data_10p=train_datagen.flow_from_directory(train_dir,target_size=IMAGE_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")
test_data=test_datagen.flow_from_directory(test_dir,target_size=IMAGE_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [14]:
#setting up callbacks 
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [15]:
#comparing 2 tensorflow models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [16]:
import tensorflow as tf
import tensorflow_hub as hub


In [17]:
def create_model(model_url,num_classes=10): # this function takes the tensorflow hub model and turn it to sequential model  and it returns a uncompiled model
  feature_extractor_layer=hub.KerasLayer(model_url,trainable=False,input_shape=IMAGE_SHAPE+(3,)) # this is freeze the already learned patterns 
  model=tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes,activation="softmax",name="output_layer")                             
  ])
  return model 

#creating and test  Resnet model and 

In [18]:
#create Resnet model 
resent_model=create_model(resnet_url,num_classes=train_data_10p.num_classes)#=10


In [10]:
resent_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23564800  
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [19]:
resent_model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [20]:
resent_model.fit(train_data_10p,epochs=5,steps_per_epoch=len(train_data_10p), validation_data=test_data,validation_steps=len(test_data),callbacks=[create_tensorboard_callback(dir_name="tensorflwo-hub", experiment_name="resnet50v2")])

Saving TensorBoard log files to: tensorflwo-hub/resnet50v2/20220517-165351
Epoch 1/5
24/24 [==============================] - 436s 19s/step - loss: 1.8529 - accuracy: 0.3667 - val_loss: 1.1562 - val_accuracy: 0.6056
Epoch 2/5
24/24 [==============================] - 431s 19s/step - loss: 0.8635 - accuracy: 0.7347 - val_loss: 0.8282 - val_accuracy: 0.7388
Epoch 3/5
24/24 [==============================] - 481s 21s/step - loss: 0.5962 - accuracy: 0.8293 - val_loss: 0.7453 - val_accuracy: 0.7564
Epoch 4/5
24/24 [==============================] - 429s 18s/step - loss: 0.4556 - accuracy: 0.8840 - val_loss: 0.6973 - val_accuracy: 0.7728
Epoch 5/5
24/24 [==============================] - 428s 18s/step - loss: 0.3734 - accuracy: 0.9120 - val_loss: 0.6746 - val_accuracy: 0.7812


In [21]:
resent_model.evaluate(test_data)

79/79 [==============================] - 329s 4s/step - loss: 0.6746 - accuracy: 0.7812


[0.6745948195457458, 0.7811999917030334]